In [ ]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2014test-A.txt',
    './dataset/train/twitter-2015train-A.txt'
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 載入 GloVe 詞嵌入
embeddings_index = {}
with open(r'C:\Users\jk121\Documents\Code\LargeData\glove.twitter.27B.25d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

In [3]:
from keras.layers import Flatten

# 建立單一 CNN 模型並加入 Flatten 層
cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 2s 4ms/step - loss: 0.9992 - accuracy: 0.4906 - val_loss: 0.9486 - val_accuracy: 0.5208
Epoch 2/10
301/301 [==============================] - 1s 3ms/step - loss: 0.9100 - accuracy: 0.5553 - val_loss: 0.9060 - val_accuracy: 0.5532
Epoch 3/10
301/301 [==============================] - 1s 3ms/step - loss: 0.8595 - accuracy: 0.5874 - val_loss: 0.8632 - val_accuracy: 0.5873
Epoch 4/10
301/301 [==============================] - 1s 3ms/step - loss: 0.8305 - accuracy: 0.6086 - val_loss: 0.8500 - val_accuracy: 0.5981
Epoch 5/10
301/301 [==============================] - 1s 3ms/step - loss: 0.8145 - accuracy: 0.6199 - val_loss: 0.8424 - val_accuracy: 0.5977
Epoch 6/10
301/301 [==============================] - 1s 3ms/step - loss: 0.7986 - accuracy: 0.6296 - val_loss: 0.8369 - val_accuracy: 0.6035
Epoch 7/10
301/301 [==============================] - 1s 2ms/step - loss: 0.7857 - accuracy: 0.6387 - val_loss: 0.8348 - val_accuracy: 0.6039
Epoch 

In [4]:
# 單一 LSTM 模型 (Single LSTM)

from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 19s 58ms/step - loss: 0.9542 - accuracy: 0.5062 - val_loss: 0.8791 - val_accuracy: 0.6022
Epoch 2/10
301/301 [==============================] - 17s 57ms/step - loss: 0.8882 - accuracy: 0.5787 - val_loss: 0.8952 - val_accuracy: 0.5910
Epoch 3/10
301/301 [==============================] - 17s 57ms/step - loss: 0.8589 - accuracy: 0.5944 - val_loss: 0.8201 - val_accuracy: 0.6247
Epoch 4/10
301/301 [==============================] - 17s 56ms/step - loss: 0.8405 - accuracy: 0.6111 - val_loss: 0.8119 - val_accuracy: 0.6205
Epoch 5/10
301/301 [==============================] - 17s 57ms/step - loss: 0.8306 - accuracy: 0.6085 - val_loss: 0.8289 - val_accuracy: 0.6085
Epoch 6/10
301/301 [==============================] - 17s 57ms/step - loss: 0.8158 - accuracy: 0.6229 - val_loss: 0.8075 - val_accuracy: 0.6500
Epoch 7/10
301/301 [==============================] - 18s 60ms/step - loss: 0.8061 - accuracy: 0.6294 - val_loss: 0.7809 - val_accuracy:

In [5]:
# CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 9s 23ms/step - loss: 0.9796 - accuracy: 0.5012 - val_loss: 0.9059 - val_accuracy: 0.5619
Epoch 2/10
301/301 [==============================] - 6s 22ms/step - loss: 0.8982 - accuracy: 0.5674 - val_loss: 0.8681 - val_accuracy: 0.5964
Epoch 3/10
301/301 [==============================] - 7s 22ms/step - loss: 0.8613 - accuracy: 0.5915 - val_loss: 0.8311 - val_accuracy: 0.6122
Epoch 4/10
301/301 [==============================] - 6s 21ms/step - loss: 0.8384 - accuracy: 0.6042 - val_loss: 0.8194 - val_accuracy: 0.6234
Epoch 5/10
301/301 [==============================] - 6s 21ms/step - loss: 0.8222 - accuracy: 0.6221 - val_loss: 0.8112 - val_accuracy: 0.6272
Epoch 6/10
301/301 [==============================] - 6s 20ms/step - loss: 0.8085 - accuracy: 0.6271 - val_loss: 0.8101 - val_accuracy: 0.6164
Epoch 7/10
301/301 [==============================] - 6s 20ms/step - loss: 0.8004 - accuracy: 0.6308 - val_loss: 0.7914 - val_accuracy: 0.6301

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

76/76 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

           0       0.49      0.53      0.51       921
           1       0.00      0.00      0.00       341
           2       0.56      0.70      0.62      1144

    accuracy                           0.54      2406
   macro avg       0.35      0.41      0.38      2406
weighted avg       0.46      0.54      0.49      2406



c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# 單一 3 層 CNN 和 LSTM 網絡

three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 6s 12ms/step - loss: 0.9663 - accuracy: 0.5062 - val_loss: 0.8829 - val_accuracy: 0.5794
Epoch 2/10
301/301 [==============================] - 3s 11ms/step - loss: 0.8741 - accuracy: 0.5808 - val_loss: 0.8531 - val_accuracy: 0.6118
Epoch 3/10
301/301 [==============================] - 3s 11ms/step - loss: 0.8156 - accuracy: 0.6225 - val_loss: 0.8092 - val_accuracy: 0.6359
Epoch 4/10
301/301 [==============================] - 3s 11ms/step - loss: 0.7869 - accuracy: 0.6383 - val_loss: 0.8057 - val_accuracy: 0.6309
Epoch 5/10
301/301 [==============================] - 3s 10ms/step - loss: 0.7454 - accuracy: 0.6654 - val_loss: 0.8023 - val_accuracy: 0.6496
Epoch 6/10
301/301 [==============================] - 3s 10ms/step - loss: 0.7008 - accuracy: 0.6896 - val_loss: 0.8030 - val_accuracy: 0.6355
Epoch 7/10
301/301 [==============================] - 3s 10ms/step - loss: 0.6587 - accuracy: 0.7155 - val_loss: 0.8365 - val_accuracy: 0.6446

In [8]:
# 多重 CNN 和 LSTM 網絡

multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 8s 20ms/step - loss: 0.9308 - accuracy: 0.5315 - val_loss: 0.8537 - val_accuracy: 0.6014
Epoch 2/10
301/301 [==============================] - 6s 19ms/step - loss: 0.8474 - accuracy: 0.5983 - val_loss: 0.8252 - val_accuracy: 0.6018
Epoch 3/10
301/301 [==============================] - 6s 19ms/step - loss: 0.7812 - accuracy: 0.6455 - val_loss: 0.7756 - val_accuracy: 0.6554
Epoch 4/10
301/301 [==============================] - 6s 19ms/step - loss: 0.7399 - accuracy: 0.6758 - val_loss: 0.7595 - val_accuracy: 0.6600
Epoch 5/10
301/301 [==============================] - 6s 19ms/step - loss: 0.7054 - accuracy: 0.6918 - val_loss: 0.7767 - val_accuracy: 0.6571
Epoch 6/10
301/301 [==============================] - 6s 19ms/step - loss: 0.6703 - accuracy: 0.7138 - val_loss: 0.7686 - val_accuracy: 0.6621
Epoch 7/10
301/301 [==============================] - 5s 18ms/step - loss: 0.6361 - accuracy: 0.7288 - val_loss: 0.8316 - val_accuracy: 0.6484

In [9]:
from keras.layers import Bidirectional

# 單一 3 層 CNN 和雙向 LSTM 網絡

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 10s 19ms/step - loss: 0.9448 - accuracy: 0.5185 - val_loss: 0.8958 - val_accuracy: 0.5752
Epoch 2/10
301/301 [==============================] - 5s 16ms/step - loss: 0.8457 - accuracy: 0.5960 - val_loss: 0.8213 - val_accuracy: 0.6222
Epoch 3/10
301/301 [==============================] - 5s 18ms/step - loss: 0.7891 - accuracy: 0.6318 - val_loss: 0.8068 - val_accuracy: 0.6301
Epoch 4/10
301/301 [==============================] - 6s 19ms/step - loss: 0.7522 - accuracy: 0.6595 - val_loss: 0.7888 - val_accuracy: 0.6409
Epoch 5/10
301/301 [==============================] - 6s 19ms/step - loss: 0.7144 - accuracy: 0.6865 - val_loss: 0.8506 - val_accuracy: 0.6122
Epoch 6/10
301/301 [==============================] - 6s 18ms/step - loss: 0.6773 - accuracy: 0.7090 - val_loss: 0.7962 - val_accuracy: 0.6517
Epoch 7/10
301/301 [==============================] - 6s 18ms/step - loss: 0.6232 - accuracy: 0.7374 - val_loss: 0.8366 - val_accuracy: 0.654

In [10]:
# 多重 CNN 和雙向 LSTM 網絡

multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 15s 37ms/step - loss: 0.9241 - accuracy: 0.5359 - val_loss: 0.8500 - val_accuracy: 0.5910
Epoch 2/10
301/301 [==============================] - 11s 36ms/step - loss: 0.8308 - accuracy: 0.6085 - val_loss: 0.7977 - val_accuracy: 0.6355
Epoch 3/10
301/301 [==============================] - 10s 34ms/step - loss: 0.7775 - accuracy: 0.6452 - val_loss: 0.7722 - val_accuracy: 0.6446
Epoch 4/10
301/301 [==============================] - 10s 34ms/step - loss: 0.7345 - accuracy: 0.6739 - val_loss: 0.7682 - val_accuracy: 0.6496
Epoch 5/10
301/301 [==============================] - 10s 34ms/step - loss: 0.7001 - accuracy: 0.6907 - val_loss: 0.7721 - val_accuracy: 0.6413
Epoch 6/10
301/301 [==============================] - 10s 33ms/step - loss: 0.6607 - accuracy: 0.7146 - val_loss: 0.7958 - val_accuracy: 0.6309
Epoch 7/10
301/301 [==============================] - 10s 33ms/step - loss: 0.6177 - accuracy: 0.7358 - val_loss: 0.8184 - val_accuracy: